In [169]:
library(bigrquery)
library(dplyr)



# Store the project ID
projectid = "spot-pet-production"

# Set your query
sql <- "SELECT * FROM `spot-pet-production.funnel_journeys.all_funnel_data_view_oc1`"

# Run the query and store the data in a tibble
tb <- bq_project_query(projectid, sql)

# Print 10 rows of the data
fulldata2<-  bq_table_download(tb)

install.packages("ChannelAttribution")
library("ChannelAttribution")

install.packages("ChannelAttribution")
library("ChannelAttribution")

#fulldata2 <-read.csv("Funnel_export.csv", header = TRUE, stringsAsFactors=FALSE)
toclean <- fulldata2[,c("customer_link","revenue", "event_step", "channel", "event_time")]
toclean$channel <- trimws(toclean$channel)
toclean <- dplyr::arrange(toclean,customer_link,event_step)
toclean[is.na(toclean)] <- 0
toclean2 <- toclean
#toclean2$event_time <- as.Date(toclean2$event_time , "%m/%d/%Y")
dates <- levels( factor(toclean2$event_time ) )

Rall <- NULL

for (p in dates) {
  toclean <-filter(toclean2, event_time == p)
fullstring <- ""
revsum = 0
paths <- data.frame( 
  stringsAsFactors=FALSE)
revenue <- data.frame( 
  stringsAsFactors=FALSE)

c_status <- data.frame( 
  stringsAsFactors=FALSE)

for (row in 1:nrow(toclean)) {
  #print(row)
  row1 = row
  number1 = toclean[row1,"event_step"]
  row2 = row+1
  number2 <- toclean[row2,"event_step"]
  check = number2-number1
  
  
  if (check == 1 & is.na(check) == FALSE) {
    string <- toclean[row1,"channel"]
    fullstring <- paste(fullstring, toclean[row1,"channel"], sep=" > ")
    revsum = revsum + toclean[row1,"revenue"]
  } else {
    #c_status1 = toclean[row1,"customer_status"]
    revsum = revsum + toclean[row1,"revenue"]
    string <- toclean[row1,"channel"]
    fullstring <- paste(fullstring, toclean[row1,"channel"], sep=" > ")
    fullstring <- as.character(fullstring)
    paths <- rbind(paths, fullstring,  stringsAsFactors = FALSE)
    fullstring <- ""
    revenue <- rbind(revenue, revsum,  stringsAsFactors = FALSE)
    revsum = 0
    #c_status <- rbind(c_status, c_status1,  stringsAsFactors = FALSE)
    
  }
  
}
    
 # 
    everything <-cbind(paths, revenue)
    names(everything)[1] <- "path"
    names(everything)[2] <- "total_conversion_value"
    everything$path <- sub('.', '', everything$path)
    everything$path <- sub('.', '', everything$path)
    everything$path <- sub('.', '', everything$path)
    everything$path <- as.factor(everything$path)
    everything1 <- everything %>% group_by(path) %>% summarize(total_conversions=n(), total_conversion_value = sum(total_conversion_value))
    everything1$total_conversions <- everything1$total_conversion_value
    H <- heuristic_models(everything1, 'path', 'total_conversions', var_value='total_conversion_value')
    M <- markov_model(everything1, 'path', 'total_conversions', var_value='total_conversion_value', order = 1) 
    R1 <- merge(H, M, by='channel_name')
    R1 <- dplyr::arrange(R1,channel_name)
    colnames(R1)[8] <- "markov_conversions"
    colnames(R1)[9] <- "markov_value"
    R1$date <- p
    Rall <- rbind(Rall, R1)
    
    
}


Rall$first_touch_value <- NULL
Rall$last_touch_value <- NULL
Rall$linear_touch_value <- NULL
Rall$markov_value <- NULL
Rall$date <- as.Date(Rall$date , "%Y-%m-%d")


projectid = "spot-pet-production"
dataset_name <- 'multi_touch_att'
partnerize_table = bq_table(project = projectid, dataset = dataset_name, table = 'policy_markov_journey')
bq_table_upload(x=partnerize_table, values= Rall, create_disposition='CREATE_IF_NEEDED', write_disposition='WRITE_TRUNCATE')





Auto-refreshing stale OAuth token.

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)




Number of simulations: 100000 - Convergence reached: 3.42% < 5.00%

Percentage of simulated paths that successfully end before maximum number of steps (19) is reached: 99.99%


Number of simulations: 100000 - Convergence reached: 2.73% < 5.00%

Percentage of simulated paths that successfully end before maximum number of steps (15) is reached: 99.86%


Number of simulations: 100000 - Convergence reached: 2.51% < 5.00%

Percentage of simulated paths that successfully end before maximum number of steps (27) is reached: 99.99%


Number of simulations: 100000 - Convergence reached: 2.84% < 5.00%

Percentage of simulated paths that successfully end before maximum number of steps (10) is reached: 99.93%


Number of simulations: 100000 - Convergence reached: 2.56% < 5.00%

Percentage of simulated paths that successfully end before maximum number of steps (10) is reached: 99.91%


Number of simulations: 100000 - Convergence reached: 3.31% < 5.00%

Percentage of simulated paths that successfull